### IMPORTS

In [46]:
import json
import base64
import hashlib
from web3 import Web3

### LOAD DATA FROM EXTERNAL FILES

In [2]:
with open('settings.json') as json_file:
    settings = json.load(json_file)

In [71]:
with open('identifier.json') as json_file:
    device = json.load(json_file)

In [3]:
with open('latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [4]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [5]:
blockchain = gateway(settings['gateways']['blockchain'])

In [6]:
ipfs = gateway(settings['gateways']['ipfs'])

### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [7]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [8]:
keys = keypair(settings['keys'])

### HASH DEVICE PARAMS FOR IDENTIFIER

In [74]:
print(device)

{'Manufacturer': 'Beispiel GmbH', 'Name': 'Example-Device', 'SerialNumber': 'D1.0', 'Type': 'example device', 'ID': '2071c7736acd16f6cea3727d3b7ecde53f4c2e97b421f3550248e19d7309c636', 'Category': 'infrastructure', 'SecureBoot': False, 'Firmware': {'Version': '1.0', 'URL': 'https://192.168.102.94:10000/FirmwareInfo'}, 'ClientSoftware': {'Version': '', 'URL': ''}, 'Updates': {'AutomaticUpdates': False, 'EndOfLife': '2021-01-01T00:00:00'}, 'Cryptography': {'Software': {'IoTAGKey': True, 'KeyStore': True, 'Algorithms': ['RSASSA-PSS', 'SHA-256', 'TLS_AES_128_GCM_SHA256', 'TLS_AES_256_GCM_SHA384', 'TLS_CHACHA20_POLY1305_SHA256', 'aes128-ctr', 'aes192-ctr', 'aes256-ctr', 'ecdsa-sha2-nistp256', 'ecdsa-sha2-nistp384', 'ecdsa-sha2-nistp521', 'ssh-rsa', 'ssh-dss', 'ecdh-sha2-nistp256', 'ecdh-sha2-nistp384', 'ecdh-sha2-nistp521', 'diffie-hellman-group-exchange-sha256', 'hmac-sha2-256,hmac-sha2-512']}, 'Hardware': {'IoTAGKey': False, 'KeyStore': False, 'Algorithms': []}}, 'Connectivity': {'IEEE802_

In [75]:
def encode(data):
    
    # CONVERT TO BYTES
    to_bytes = str.encode(str(data))
    
    # HASH ENCODED DATA
    hashed = hashlib.sha224(to_bytes).hexdigest()
    
    return hashed

In [76]:
identifier = encode(device)

In [77]:
identifier

'd7fdb8228681c9999294d997a0f21f58820115d5c830db4efe0fc7e9'

### CONNECT TO CHAIN VIA WEBSOCKET

In [9]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [40]:
if web3.isConnected():
    print('Successfully connected to gateway via websocket!')

Successfully connected to gateway via websocket!


### CONSTRUCT MANAGER CONTRACT

In [19]:
class manager:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, func):
        return self.contract.functions[func]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': keys.public,
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.eth.generateGasPrice()
            tx['nonce'] = web3.eth.getTransactionCount(keys.public)

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = keys.private
            )

            # SEND THE TRANSACTION
            web3.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return 'success'
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error

### CONTRACT REFERENCES

In [20]:
user_manager = manager(latest['usermanager'])

In [21]:
device_manager = manager(latest['devicemanager'])

In [22]:
task_manager = manager(latest['taskmanager'])

In [23]:
token_manager = manager(latest['tokenmanager'])

### TESTING

In [27]:
# token_manager.read('price')

In [28]:
#token_manager.write({
#    'func': 'init',
#    'params': [5, task_manager.address]
#})